In [1]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import FunctionTransformer
from sentence_transformers import SentenceTransformer
import joblib

/Users/ethanevans/Library/Python/3.8/lib/python/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
INSIDER_TERMS = [
    "kek",
    "kekistan",
    "1488",
    "14 88",
    "bap", # Bronze Age Pervert
    "blood and soil",
    "gae", # Globalist American Empire
    "zog", # Zionist Occupied Government
    "wypipo",
    "soyjak",
    "femoid",
    "zog",
    "rino",
    "kang",
    "kangz",
    "negroid",
    "fiume",
    "gnc", # Global N***** Communism
]

# --- MiniLM Transformer ---
class MiniLMTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return self.model.encode(X['text'].tolist(), convert_to_numpy=True)
    

# --- Insider Terms ---
class InsiderTermsTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, terms=None):
        self.terms = [term.lower() for term in (terms or INSIDER_TERMS)]

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        texts = X['text'].str.lower().tolist()
        features = []

        for text in texts:
            features.append([int(term in text) for term in self.terms])

        return np.array(features)
    

class Pre2018Selector(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[['pre_2018']].values.astype(float)


def select_text_column(X):
    return X['text']


def create_model(dirty=False):
    # Load data
    df = pd.read_csv('tweets.csv')
    
    # Basic cleaning if needed
    if dirty:
        df = df.dropna(subset=['text'])
        df = df[df['text'].str.strip() != '']

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        df[['text']], df['extreme'], test_size=0.2, random_state=42
    )

    # Combine features
    combined_features = FeatureUnion([
        ('tfidf', Pipeline([
            ('selector', FunctionTransformer(select_text_column, validate=False)),
            ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1, 2)))
        ])),
        ('minilm', MiniLMTransformer()),
        ('insider_terms', InsiderTermsTransformer()),
        #('pre_2018', Pre2018Selector())
    ])

    # Full pipeline with non-linear SVM
    model = Pipeline([
        ('features', combined_features),
        ('clf', SVC(kernel='rbf', class_weight='balanced'))
    ])

    # Train the model
    model.fit(X_train, y_train)

    # Evaluate
    y_pred = model.predict(X_test)
    print("\nClassification Report (TF-IDF + MiniLM + RBF-SVM):\n")
    print(classification_report(y_test, y_pred, target_names=['normal', 'extreme']))

    # Save trained model
    joblib.dump(model, 'model.pkl')

In [ ]:
# Run model creation
create_model(dirty=True)

# 2 min


Classification Report (TF-IDF + MiniLM + RBF-SVM):

              precision    recall  f1-score   support

      normal       0.95      0.95      0.95       610
     extreme       0.91      0.91      0.91       324

    accuracy                           0.94       934
   macro avg       0.93      0.93      0.93       934
weighted avg       0.94      0.94      0.94       934

